In [3]:
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from datasets import Dataset, load_from_disk
import torch.nn.functional as F

import numpy as np
import pandas as pd

In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [5]:
path_final_test = './data/final_test/'

In [6]:
BATCH_SIZE = 128

In [7]:
preprocess = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406],  std=[0.229, 0.224, 0.225])])

In [8]:
test_dataset = ImageFolder(path_final_test, preprocess)

In [9]:
test_dataset

Dataset ImageFolder
    Number of datapoints: 1751
    Root location: ./data/final_test/
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=True)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [10]:
testDataLoader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [11]:
next(iter(testDataLoader))[1]

tensor([0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
        1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0,
        0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0,
        0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0,
        1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
        0, 1, 1, 1, 0, 1, 0, 1])

In [12]:
resnet34 = models.resnet34(weights=None)
class CustomFC(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.fc1 = nn.Linear(in_features=in_features, out_features=out_features)
        
    def forward(self, x):
        return self.fc1(x), x
    
resnet34.fc = CustomFC(512,8)
resnet34.to(device)

resnet34.load_state_dict(torch.load('model_best_resnet34.pt', map_location=device, weights_only=False))

<All keys matched successfully>

In [13]:
class AutoEncoder(torch.nn.Module):
    def __init__(self, n, m, p, b):
        super().__init__()
        self.e1 = torch.nn.Linear(n, m)
        self.e2 = torch.nn.Linear(m, p)
        self.e3 = torch.nn.Linear(p, b)
        self.dec3 = torch.nn.Linear(b, p)
        self.dec2 = torch.nn.Linear(p, m)
        self.dec1 = torch.nn.Linear(m, n)
        
    def forward(self, x):
        x = self.e1(x)
        x = self.e2(F.relu(x))
        encoded = self.e3(F.relu(x))
        x = self.dec3(encoded)
        x = self.dec2(F.relu(x))
        x = self.dec1(F.relu(x))
        return x

In [17]:
del model_auto

model_auto = AutoEncoder(512, 256, 128, 32)
model_auto.to(device)
model_auto.load_state_dict(torch.load('best_model_autoenc32_resnet34.pt', map_location=device, weights_only=False))

<All keys matched successfully>

In [18]:
def sep_mod_acc(model1, model2, dataloader_test, er_mean_rat):
    loss_fn = torch.nn.MSELoss(reduction='none')
    model1.eval()
    model2.eval()
    trg = []
    pred = []
    acc_test = 0.0
    tot_test = 0.0
    big_ones = 0.0
    tot_positive = 0.0
    with torch.no_grad():
        for itms in dataloader_test:
            x=itms[0]
            target=itms[1]
            x = x.to(device)
            target = target.to(device)
            outputs, emb = model1(x)
            x_pr = model2(emb)
            error = torch.mean(loss_fn(x_pr, emb), dim=1)/0.0026712
            acc_test += torch.sum(torch.where(error>er_mean_rat,0,1) == target).item()
            tot_test += target.size(0)
            
            
            # outputs = outputs.sigmoid().max(dim=1).values

        #     big_ones += torch.sum(outputs.round()).item()
        #     pred.extend(outputs.round().cpu().tolist())
        #     trg.extend(target.cpu().tolist())
        #     tot_test += target.size(0)
        #     tot_positive += torch.sum(target).item()
        # print(f'{er_mean_rat}Test accuracy: {round(acc_test/tot_test*100,2)}%')
        # print(f'Higher than 50%: {round(big_ones/tot_test*100,2)}%')
        # print(f'Non-anomalies to total: {round(tot_positive/tot_test*100,2)}%')
        
    return round(acc_test/tot_test*100,2)#, round(big_ones/tot_test*100,2), pd.DataFrame({'true':trg, 'pred':pred})

In [19]:
res = []
for i in range(1,30):
    res.append(sep_mod_acc(resnet34, model_auto, testDataLoader, i*.1))
max(res)


60.14